In [69]:
import little_mallet_wrapper as lmw
import seaborn
import spacy
import pandas as pd
import random
from time import process_time
from nltk import WordNetLemmatizer
from multiprocessing import Pool

## Cleaning objectives (before mallet)
- remove formatting (i.e. \n)
- lemmatize

In [83]:
#bring in data
inPath = "/shared/3/projects/benlitterer/localNews/mergedArticles.tsv"
articlesDf = pd.read_csv(inPath, sep="\t")

/opt/anaconda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3418: DtypeWarning: Columns (1,2,6,9,10,11,12,13,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,38,39,40,41) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [84]:
numRows = len(articlesDf)

In [85]:
#806,159 articles
sampled = random.sample(list(range(1, numRows)), int(numRows/4))
len(sampled)

806159

In [87]:
articlesDf = articlesDf.iloc[sampled]

In [88]:
articlesDf

,Unnamed: 0,article_id,sourcedomain_id,date,title,content,url_x,fips,source,description,...,predrt_0,predrt_12,predrt_3,national,id,author,url,published,published_utc,collection_utc
2973366,2973366,NaN,NaN,2020-10-02 00:00:00,WHO urges against harsh restrictions amid grow...,"MOSCOW , October 2 .\n/TASS/ .\nThe World Heal...",NaN,NaN,tass,NaN,...,NaN,NaN,NaN,True,tass--2020-10-02--WHO urges against harsh rest...,NaN,https://tass.com/world/1208071,"Fri, 02 Oct 2020 20:02:33 +0300",1.601683e+09,1.601691e+09
2241142,2241142,NaN,NaN,2020-04-25 00:00:00,Listen: New Audio Evidence Emerges Against Bid...,A new report from The Intercept ’ s Ryan Grim ...,NaN,NaN,sputnik,NaN,...,NaN,NaN,NaN,True,sputnik--2020-04-25--Listen: New Audio Evidenc...,NaN,https://sputniknews.com/us/202004251079087667-...,"Sat, 25 Apr 2020 00:59:58 +0000",1.587791e+09,1.587777e+09
340177,340177,mountainxpress--2020-08-13--Letter: Artsville ...,mountainxpress-mountainx.com,2020-08-13,Letter: Artsville fails to impress,Your city is a prissy little bourgeois phony a...,https://mountainx.com/opinion/letter-artsville...,37021.0,mountainxpress,"Entertainment, news, arts and events for Ashev...",...,30.96,45.38,23.65,False,NaN,NaN,NaN,NaN,NaN,NaN
2883919,2883919,NaN,NaN,2020-10-25 00:00:00,Air Force vet would be California's first Blac...,An Air Force veteran in Southern California wo...,NaN,NaN,foxnews,NaN,...,NaN,NaN,NaN,True,foxnews--2020-10-25--Air Force vet would be Ca...,Brie Stimson,http://feeds.foxnews.com/~r/foxnews/politics/~...,"Sun, 25 Oct 2020 10:09:49 GMT",1.603635e+09,1.603628e+09
721331,721331,harrodsburgherald--2021-01-27--County OKs Rais...,harrodsburgherald-harrodsburgherald.com,2021-01-27,County OKs Raising Fees At Animal Shelter,The Mercer County Fiscal Court voted to increa...,https://www.harrodsburgherald.com/2021/01/27/c...,21167.0,harrodsburgherald,"Serves Mercer County by providing local news, ...",...,32.34,45.32,22.34,False,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2969459,2969459,NaN,NaN,2020-07-21 00:00:00,"Putin commends TASS.com head Golovina, senior ...",Among those awarded by the president are TASS ...,NaN,NaN,tass,NaN,...,NaN,NaN,NaN,True,tass--2020-07-21--Putin commends TASS.com head...,NaN,https://tass.com/society/1180823,"Tue, 21 Jul 2020 14:57:55 +0300",1.595358e+09,1.595336e+09
947521,947521,thekennebecjournal--2021-07-31--NFL notebook: ...,thekennebecjournal-centralmaine.com,2021-07-31,NFL notebook: Patriots QB Stidham reportedly h...,"According to the NFL Network, Patriots backup ...",https://www.centralmaine.com/2021/07/31/nfl-no...,23011.0,thekennebecjournal,"Coverage of local, city wide news with some co...",...,33.72,44.69,21.60,False,NaN,NaN,NaN,NaN,NaN,NaN
1650096,1650096,NaN,NaN,2020-03-09 00:00:00,Man shot dead in row over parking space,A fight over a parking space led to the fatal ...,NaN,NaN,theindependent,NaN,...,NaN,NaN,NaN,True,theindependent--2020-03-09--Man shot dead in r...,Agencies,https://www.independent.co.uk/news/world/ameri...,"Mon, 09 Mar 2020 09:56:02 GMT",1.583762e+09,1.583754e+09
2833015,2833015,NaN,NaN,2020-01-26 00:00:00,Scots judo team brave nippy -40C temperatures ...,"As Burns Suppers go , it would be enough to le...",NaN,NaN,thedailyrecord,NaN,...,NaN,NaN,NaN,True,thedailyrecord--2020-01-26--Scots judo team br...,reporters@dailyrecord.co.uk (Norman Silvester),https://www.dailyrecord.co.uk/news/scottish-ne...,"Sun, 26 Jan 2020 04:30:00 +0000",1.580031e+09,1.580041e+09


## Lemmatize (Don't worry about this for now) 

In [74]:
"""
#subset it for now just to make sure things run alright
articlesDf = pd.concat([articlesDf.iloc[0:4000], articlesDf.iloc[3224636-4000:3224636]]).reset_index(drop=True) 

#use spacy to clean the data (as one spacy object??)
nlp = spacy.load("en_core_web_sm")

docs = list(articlesDf["content"])
docs = nlp.pipe(docs, disable=["parser", "ner"],batch_size=100, n_process=12)


# for all documents this would take ~30 hours? 
# can multiprocess and implement memoization 
t1_start = process_time()

#Lemmatize
memos = {}
lemmaDocs = []
i = 0
for doc in docs: 
    #dat = doc.to_array(["LEMMA"])
    
    #def getLemma(item, pos): 
    #    if item not in memos: 
    #        return item.lemma 
    #     
    
    lemmaDoc = " ".join([item.lemma_ for item in doc])
    lemmaDocs.append(lemmaDoc)
    
    
    if i % 200 == 0: 
        print(i)
    i += 1
    
t1_stop = process_time()
print("Elapsed time: ") 
print(t1_stop - t1_start)

articlesDf["lemmatizedContent"] = lemmaDocs
"""

'\n#subset it for now just to make sure things run alright\narticlesDf = pd.concat([articlesDf.iloc[0:4000], articlesDf.iloc[3224636-4000:3224636]]).reset_index(drop=True) \n\n#use spacy to clean the data (as one spacy object??)\nnlp = spacy.load("en_core_web_sm")\n\ndocs = list(articlesDf["content"])\ndocs = nlp.pipe(docs, disable=["parser", "ner"],batch_size=100, n_process=12)\n\n\n# for all documents this would take ~30 hours? \n# can multiprocess and implement memoization \nt1_start = process_time()\n\n#Lemmatize\nmemos = {}\nlemmaDocs = []\ni = 0\nfor doc in docs: \n    #dat = doc.to_array(["LEMMA"])\n    \n    #def getLemma(item, pos): \n    #    if item not in memos: \n    #        return item.lemma \n    #     \n    \n    lemmaDoc = " ".join([item.lemma_ for item in doc])\n    lemmaDocs.append(lemmaDoc)\n    \n    \n    if i % 200 == 0: \n        print(i)\n    i += 1\n    \nt1_stop = process_time()\nprint("Elapsed time: ") \nprint(t1_stop - t1_start)\n\narticlesDf["lemmatizedCo

## *Very* quick and dirty cleaning

In [101]:
import tqdm 
#only took ~3 minutes 

def process(inStr): 
    return lmw.process_string(str(inStr))

with Pool(12) as p: 
    cleaned = list(tqdm.tqdm(p.imap(process, articlesDf["content"].tolist()), total=len(articlesDf)))

100%|██████████| 806159/806159 [02:50<00:00, 4720.53it/s]


## Sanity Checks 

In [ ]:
len(articlesDf) == len(cleaned)

In [112]:
cleaned[6][:300]

'former hibs celtic star derek riordan gone pub business convicted drug dealer riordan NUM teamed michael mikey wright NUM take four hand bar edinburgh leith easter road watering hole previously owned another former hibs player paul kane used past football casuals four hand also targeted firebomb att'

In [113]:
articlesDf["content"].reset_index(drop=True)[6][:300]

'Former Hibs and Celtic star Derek Riordan has gone into the pub business with a convicted drug dealer .\nRiordan , 37 , has teamed up with Michael “ Mikey ” Wright , 35 , to take over the Four In Hand bar in Edinburgh ’ s Leith .\nThe Easter Road watering hole was previously owned by another former Hi'

In [116]:
len(articlesDf) == len(cleaned)

True

In [120]:
#only needed once: cleaned = lemmatized
articlesDf["contentCleaned"] = cleaned 

In [125]:
articlesDf[["source", "contentCleaned"]].to_csv("/shared/3/projects/benlitterer/localNews/malletInputQuarter.tsv", sep="\t", index=True)